### Chains with multiple inputs

In [1]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

# Basic chain with single input
prompt_template = ChatPromptTemplate.from_messages([("user", "Tell me a joke about {input}")])
chain = prompt_template | llm | StrOutputParser()
chain.invoke({"input": "a parrot"})

'Why did the parrot get fired from the call center?\n\nBecause every time someone asked for technical support, he\'d squawk, "Polly wants a cracker! And maybe a new motherboard!"'

In [2]:
# Chain with multiple inputs
prompt_template = ChatPromptTemplate.from_messages([("user", "Tell me a joke about {input} in {language}")])
chain = prompt_template | llm | StrOutputParser()
chain.invoke({"input": "a parrot", "language": "german"})

'Hier ist ein Witz über einen Papagei auf Deutsch:\n\nEin Mann kommt in eine Tierhandlung und sieht einen schönen Papagei.\nEr fragt den Verkäufer: "Was kostet denn dieser Papagei?"\nDer Verkäufer antwortet: "Dieser Papagei ist etwas Besonderes, er kann nämlich sprechen. Er kostet 1000 Euro."\nDer Mann ist beeindruckt: "Wirklich? Und was kann er sagen?"\nDer Verkäufer dreht sich zum Papagei und sagt: "Na los, Hansi, sag mal was!"\nDer Papagei bleibt stumm.\nDer Mann: "Aber er sagt doch gar nichts!"\nDer Papagei blickt den Mann an und sagt: "Ich bin doch nicht blöd! Wenn ich jetzt rede, verkauft der mich doch wieder für 1000 Euro!"'

Chains can be more complex and not all sequential chains will be as simple as passing a single string as an argument and getting a single string as output for all steps in the chain

In [3]:
# This is a chain to write a review given a dish name and the experience.
prompt_review = ChatPromptTemplate.from_messages([
    ("user", "You ordered {dish_name} and your experience was {experience}. Write a review:")
])

# This is a chain to write a follow-up comment given the restaurant review.
prompt_comment = ChatPromptTemplate.from_messages([
    ("user", "Given the restaurant review: {review}, write a follow-up comment:")
])

# This is a chain to summarize a review.
prompt_summary = ChatPromptTemplate.from_messages([
    ("user", "Summarise the review in one short sentence: {comment}")
])

# This is a chain to translate a summary into German.
prompt_translation = ChatPromptTemplate.from_messages([
    ("user", "Translate the summary to german: {summary}")
])

# Create individual chains
chain_review = prompt_review | llm | StrOutputParser()
chain_comment = prompt_comment | llm | StrOutputParser()
chain_summary = prompt_summary | llm | StrOutputParser()
chain_translation = prompt_translation | llm | StrOutputParser()

# Create overall sequential chain using LCEL
def create_overall_chain():
    def review_step(inputs):
        return {"review": chain_review.invoke(inputs)}
    
    def comment_step(inputs):
        return {"comment": chain_comment.invoke(inputs)}
    
    def summary_step(inputs):
        return {"summary": chain_summary.invoke(inputs)}
    
    def translation_step(inputs):
        return {"german_translation": chain_translation.invoke(inputs)}
    
    from langchain_core.runnables import RunnableLambda
    
    overall_chain = (
        RunnableLambda(review_step) |
        RunnableLambda(comment_step) |
        RunnableLambda(summary_step) |
        RunnableLambda(translation_step)
    )
    
    return overall_chain

overall_chain = create_overall_chain()
result = overall_chain.invoke({"dish_name": "Pizza Salami", "experience": "It was awful!"})
print("Review:", result.get('review', ''))
print("Comment:", result.get('comment', ''))
print("Summary:", result.get('summary', ''))
print("German Translation:", result.get('german_translation', ''))

Review: 
Comment: 
Summary: 
German Translation: Die Rezension beschrieb die Pizza Salami als ein "absolutes Desaster" aufgrund ihres verbrannten/durchgeweichten Bodens, der fettigen Salami, des gummiartigen Käses und der wässrigen Soße, wodurch sie ungenießbar wurde.


Instead of chaining multiple chains together we can also use an LLM to decide which follow up chain is being used

In [4]:
# Define templates for different sentiment analysis approaches
positive_template = """You are an AI that focuses on the positive side of things. 
Whenever you analyze a text, you look for the positive aspects and highlight them. 
Here is the text:
{input}"""

neutral_template = """You are an AI that has a neutral perspective. You just provide a balanced analysis of the text, 
not favoring any positive or negative aspects. Here is the text:
{input}"""

negative_template = """You are an AI that is designed to find the negative aspects in a text. 
You analyze a text and show the potential downsides. Here is the text:
{input}"""

prompt_infos = [
    {
        "name": "positive",
        "description": "Good for analyzing positive sentiments",
        "prompt_template": positive_template,
    },
    {
        "name": "neutral",
        "description": "Good for analyzing neutral sentiments",
        "prompt_template": neutral_template,
    },
    {
        "name": "negative",
        "description": "Good for analyzing negative sentiments",
        "prompt_template": negative_template,
    },
]

# Create destination chains
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(prompt_template)
    chain = prompt | llm | StrOutputParser()
    destination_chains[name] = chain

print("Destination chains created:")
for name in destination_chains:
    print(f"- {name}")

Destination chains created:
- positive
- neutral
- negative


In [5]:
# Create router prompt
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

router_template = """Given a piece of text, determine which sentiment analysis approach to use. 
Here are the available options:\n""" + destinations_str + """

Return the name of the destination chain to use (positive, neutral, or negative).
Text: {input}
Destination:"""

router_prompt = ChatPromptTemplate.from_template(router_template)

# Create router chain
def router_function(inputs):
    """Route the input to the appropriate chain based on the router's decision."""
    # Get the router's decision
    router_output = (router_prompt | llm | StrOutputParser()).invoke(inputs)
    
    # Extract the destination name from the router output
    destination = router_output.strip().lower()
    
    # Default to neutral if the destination is not recognized
    if destination not in destination_chains:
        destination = "neutral"
    
    # Invoke the appropriate destination chain
    result = destination_chains[destination].invoke(inputs)
    return {"result": result, "destination": destination}

from langchain_core.runnables import RunnableLambda

router_chain = RunnableLambda(router_function)

# Test the router chain
test_input = "I ordered Pizza Salami for 9.99$ and it was awesome!"
result = router_chain.invoke({"input": test_input})
print(f"Router chose: {result['destination']}")
print(f"Result: {result['result']}")

Router chose: positive
Result: That's wonderful to hear! Let's highlight the great things in your experience:

*   **Awesome Pizza:** The most prominent positive is that the pizza was described as "awesome!" – that's fantastic!
*   **Successful Order:** You successfully ordered exactly what you wanted, "Pizza Salami."
*   **Good Value:** At just $9.99, getting an "awesome" pizza suggests you received great value for your money.

It sounds like you had a truly satisfying and delicious meal!
